In [1]:
pip install stanza

Note: you may need to restart the kernel to use updated packages.


In [2]:
import stanza
import pandas as pd

In [3]:
stanza.download('hindi')
nlp = stanza.Pipeline('hi')

2023-02-13 15:03:45 INFO: "hindi" is an alias for "hi"
2023-02-13 15:03:45 INFO: Downloading default packages for language: hi (Hindi) ...
2023-02-13 15:03:46 INFO: File exists: C:\Users\lenovo\stanza_resources\hi\default.zip
2023-02-13 15:03:51 INFO: Finished downloading models and saved to C:\Users\lenovo\stanza_resources.
2023-02-13 15:03:51 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-02-13 15:03:52 INFO: Loading these models for language: hi (Hindi):
| Processor | Package |
-----------------------
| tokenize  | hdtb    |
| pos       | hdtb    |
| lemma     | hdtb    |
| depparse  | hdtb    |

2023-02-13 15:03:52 INFO: Use device: cpu
2023-02-13 15:03:52 INFO: Loading: tokenize
2023-02-13 15:03:52 INFO: Loading: pos
2023-02-13 15:03:53 INFO: Loading: lemma
2023-02-13 15:03:53 INFO: Loading: depparse
2023-02-13 15:03:54 INFO: Done loading processors!


In [4]:
nlp = stanza.Pipeline(lang ="hi", processors = "tokenize,pos,lemma")

2023-02-13 15:03:54 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-02-13 15:03:55 INFO: Loading these models for language: hi (Hindi):
| Processor | Package |
-----------------------
| tokenize  | hdtb    |
| pos       | hdtb    |
| lemma     | hdtb    |

2023-02-13 15:03:55 INFO: Use device: cpu
2023-02-13 15:03:55 INFO: Loading: tokenize
2023-02-13 15:03:55 INFO: Loading: pos
2023-02-13 15:03:56 INFO: Loading: lemma
2023-02-13 15:03:56 INFO: Done loading processors!


In [5]:
#dictionary that contains pos tags and their explanations
pos_dict = {
'CC': 'coordinating conjunction','CD': 'cardinal digit','DT': 'determiner',
'EX': 'existential there (like: \"there is\" ... think of it like \"there exists\")',
'FW': 'foreign word','IN':  'preposition/subordinating conjunction','JJ': 'adjective \'big\'',
'JJR': 'adjective, comparative \'bigger\'','JJS': 'adjective, superlative \'biggest\'',
'LS': 'list marker 1)','MD': 'modal could, will','NN': 'noun, singular \'desk\'',
'NNS': 'noun plural \'desks\'','NNP': 'proper noun, singular \'Harrison\'',
'NNPS': 'proper noun, plural \'Americans\'','PDT': 'predeterminer \'all the kids\'',
'POS': 'possessive ending parent\'s','PRP': 'personal pronoun I, he, she',
'PRP$': 'possessive pronoun my, his, hers','RB': 'adverb very, silently,',
'RBR': 'adverb, comparative better','RBS': 'adverb, superlative best',
'RP': 'particle give up','TO': 'to go \'to\' the store.','UH': 'interjection errrrrrrrm',
'VB': 'verb, base form take','VBD': 'verb, past tense took',
'VBG': 'verb, gerund/present participle taking','VBN': 'verb, past participle taken',
'VBP': 'verb, sing. present, non-3d take','VBZ': 'verb, 3rd person sing. present takes',
'WDT': 'wh-determiner which','WP': 'wh-pronoun who, what','WP$': 'possessive wh-pronoun whose',
'WRB': 'wh-abverb where, when','QF' : 'quantifier, bahut, thoda, kam (Hindi)','VM' : 'main verb',
'PSP' : 'postposition, common in indian langs','DEM' : 'demonstrative, common in indian langs'
}

#extract parts of speech
def extract_pos(doc):
    parsed_text = {'word':[], 'pos':[], 'exp':[]}
    for sent in doc.sentences:
        for wrd in sent.words:
            if wrd.pos in pos_dict.keys():
                pos_exp = pos_dict[wrd.pos]
            else:
                pos_exp = 'NA'
            parsed_text['word'].append(wrd.text)
            parsed_text['pos'].append(wrd.xpos)
            parsed_text['exp'].append(wrd.upos)
    #return a dataframe of pos and text
    return pd.DataFrame(parsed_text)

In [78]:
def rule_based_approach(s):
    hindi_doc = nlp(s)
    df = extract_pos(hindi_doc)
    dictionary = df[['word','exp']].set_index('word').to_dict()['exp']
    dictionary['(']= "NILL"
    dictionary[')']='NILL'
    dictionary[" "]='NILL'
    
    original_array =  s.split()
    
    # CEP
    ending =""

    i=0
    while i< len(original_array):
        ending+=original_array[i]
        ending+=" "

        if(dictionary[original_array[i]]=='VERB'):
            j=i+1
            while(j< len(original_array)):
                if(dictionary[original_array[j]]=='AUX'):
                    ending+=original_array[j]
                    ending+=" "
                    j=j+1
                else:
                    break
            ending+=')'
            ending+=" "
            i=j-1
        i=i+1


    ending= ending.strip()
    
    modified_array = ending.split()
    last = len(modified_array)

    if modified_array[last-1]!=')':
        ending+=" "
        ending+=')'

    modified_array = ending.split()
    print(modified_array)
    
    # now mark CSP
    # finite verb
    starting =""
    
    for i in range(len(modified_array)):
        f=0
        if(modified_array[i]!=')'):
            if(dictionary[modified_array[i]]=='CCONJ'):
                f=1
            if(dictionary[modified_array[i]]=='SCONJ'):
                f=2
            if(dictionary[modified_array[i]]=='PRON'):
                if(modified_array[i]=='जिसने'):
                    f=2
                if(modified_array[i]=='जिसे'):
                    f=2
                if(modified_array[i]=='जो'):
                    f=2
                if(modified_array[i]=='जिनको'):
                    f=2


        if f==1:
            starting+=modified_array[i]
            starting+=" "
            starting+='('
            starting+=" "

        elif f==2:
            starting+='('
            starting+=" "
            starting+=modified_array[i]
            starting+=" "

        elif f==0:
            starting+=modified_array[i]
            starting+=" "
    
    if starting[0]!='(':
        starting= '('+" "+ starting
        
    starting = starting.strip()
    print(starting)
    
    
    # sanity checker
    strt = starting
    left=0
    right=0

    for i in range(len(strt)):
        if(strt[i]=='('):
            left=left+1
        if(strt[i]==')'):
            right=right+1

#     print(left)
#     print(right)

    diff = abs(left-right)
    if left>right:
        while(diff):
            strt+=')'
            diff=diff-1

    if right>left:
        temp_strt = strt.split()
        temp_st =""
        j=0
        
        while j<len(temp_strt):
            if dictionary[temp_strt[j]]== 'PROPN':
                if diff>0 & j-1>=0:
                    if temp_strt[j-1]!='(':
                        temp_st+='('
                        temp_st+=" "
                        diff=diff-1
            temp_st+=temp_strt[j]
            temp_st+=" "
            j=j+1
        
        strt = temp_st    
    
    
        while(diff):
            strt='('+strt
            diff=diff-1
    
    return strt
    

In [106]:
final_string = rule_based_approach("राम जो खेल रहा था नहीं आया")

['राम', 'जो', 'खेल', 'रहा', 'था', ')', 'नहीं', 'आया', ')']
( राम ( जो खेल रहा था ) नहीं आया )


In [107]:
final_string

'( राम ( जो खेल रहा था ) नहीं आया )'